# Command

It helps transform a jumble of scattered function calls into well-organized commands that are easy to manage and extend.

## Implementation

In [2]:
# take an example without the command pattern
from dataclasses import dataclass

@dataclass
class BankAccount:
    balance: int
    overdue: int = -500

    def deposit(self, amount):
        self.balance += amount

    def withdraw(self, amount):
        if self.balance - amount > self.overdue:
            self.balance -= amount
            return True
        return False

    def __str__(self):
        return f"There are ${self.balance} left."

In [59]:
# using command pattern
from abc import abstractmethod
from enum import Enum

# command interface
class Command:
    @abstractmethod
    def call(self):
        pass

# concrete command classes
class BankAccountCommand(Command):
    class Action(Enum):
        Deposit=1,
        Withdraw=2
        
    def __init__(self, account, action, amount):
        self.account = account
        self.action = action
        self.amount = amount

    def call(self):
        if self.action is self.Action.Deposit:
            self.account.deposit(self.amount)
        elif self.action is self.Action.Withdraw:
            self.account.withdraw(self.amount)
                

In [60]:
account = BankAccount(200)
print(account)
command = BankAccountCommand(account, BankAccountCommand.Action.Deposit, 300)
command.call()
print(account)
command = BankAccountCommand(account, BankAccountCommand.Action.Withdraw, 100)
command.call()
print(account)

There are $200 left.
There are $500 left.
There are $400 left.


## Undo

In [14]:
# if we need to reverse, we can add an undo function to the Commands

from abc import abstractmethod
from enum import Enum

# command interface
class Command:
    @abstractmethod
    def call(self):
        pass

    @abstractmethod
    def undo(self):
        pass

# concrete command classes
class BankAccountCommand(Command):
    class Action(Enum):
        Deposit=1,
        Withdraw=2
        
    def __init__(self, account, action, amount):
        self.account = account
        self.action = action
        self.amount = amount
        self.succeed = False

    def call(self):
        if self.action is self.Action.Deposit:
            self.account.deposit(self.amount)
            self.succeed = True
        elif self.action is self.Action.Withdraw:
            self.account.withdraw(self.amount)
            self.succeed = True

    def undo(self):
        if self.action is self.Action.Deposit:
            if self.succeed:
                self.account.withdraw(self.amount)
                self.succeed = False
        elif self.action is self.Action.Withdraw:
            if self.succeed:
                self.account.deposit(self.amount)
                self.succeed = False
                

In [15]:
account = BankAccount(200)
print(account)
command = BankAccountCommand(account, BankAccountCommand.Action.Deposit, 300)
command.call()
print(account)
command = BankAccountCommand(account, BankAccountCommand.Action.Withdraw, 100)
command.call()
print(account)
command.undo()
print(account)
command.undo()
print(account)

There are $200 left.
There are $500 left.
There are $400 left.
There are $500 left.
There are $500 left.


## Compoite Command

In [72]:
# Here, we are insterested in the scenario where someone withdraw an amount of money 
# and deposit it to another account.

class CompositeBankAccountCommand(Command):
        
    def __init__(self, commands):
        self.commands = commands

    def call(self):
        for cmd in self.commands:
            cmd.call();

    def undo(self):
        for cmd in reversed(self.commands):
            cmd.undo()

class MoneyTransferCommand(CompositeBankAccountCommand):
    def __init__(self, account_from, account_to, amount):
        super().__init__([BankAccountCommand(account_from, BankAccountCommand.Action.Withdraw, amount),
                          BankAccountCommand(account_to, BankAccountCommand.Action.Deposit, amount)])

    def call(self):
        for cmd in self.commands:
            cmd.call()
            cmd.succeeded = True
            
        

In [74]:
account1 = BankAccount(200)
account2 = BankAccount(400)
transfer = MoneyTransferCommand(account1, account2, 200)
transfer.call()
print(f"account1: {account1}, account2: {account2}")

account1: There are $0 left., account2: There are $600 left.


## Multiple Commands

In [16]:
import os
from abc import ABC, abstractmethod
from typing import TextIO

class FileReceiver:
    def __init__(self, filename: str):
        self.filename = filename
        self.file: TextIO | None = None

    def open_file(self):
        self.file = open(self.filename, 'w')
        print(f"File {self.filename} opened.")

    def write_file(self, text: str):
        if not self.file:
	         raise Exception("File is not open")
        self.file.write(text)
        print("Written to file:", text)


    def close_file(self):
        if self.file:
            self.file.close()
            print(f"File {self.filename} closed.")
        else:
            raise Exception("File is not open")

class Command(ABC):
    @abstractmethod
    def execute(self):
        pass

class OpenFileCommand(Command):
    def __init__(self, file_receiver: FileReceiver):
        self.file_receiver = file_receiver

    def execute(self):
        self.file_receiver.open_file()

class WriteFileCommand(Command):
    def __init__(self, file_receiver: FileReceiver, text: str):
        self.file_receiver = file_receiver
        self.text = text

    def execute(self):
        self.file_receiver.write_file(self.text)

class CloseFileCommand(Command):
    def __init__(self, file_receiver: FileReceiver):
        self.file_receiver = file_receiver

    def execute(self):
        self.file_receiver.close_file()

class FileOperation:
    def __init__(self, commands: list[Command]):
        self.commands = commands

    def execute_commands(self):
        for command in self.commands:
            command.execute()

file_receiver = FileReceiver("example.txt")
commands = [
    OpenFileCommand(file_receiver),
    WriteFileCommand(file_receiver, "Hello, this is a test."),
    CloseFileCommand(file_receiver)
]

file_operations = FileOperation(commands)
file_operations.execute_commands()

File example.txt opened.
Written to file: Hello, this is a test.
File example.txt closed.


## Command Query Separation

In [120]:
from enum import Enum

@dataclass
class CharacterCommand:
    class Action(Enum):
        setby = 1,
        increseby = 2, 
        decreaseby = 3  
    action: Action
    indicator : Enum
    amount: int
    
@dataclass
class CharacterQuery:
    indicator: Enum

In [170]:
@dataclass
class Character:
    class CharacterIndicator(Enum):
        Health = 1,
        Power = 2

    _name: str
    _health: int
    _power: int
    
    def process_command(self, cmd: CharacterCommand):
        
        # 
        indicator = {self.CharacterIndicator.Health: self._health, \
                     self.CharacterIndicator.Power: self._power}

        if cmd.action == CharacterCommand.Action.setby:
            indicator[cmd.indicator] = cmd.amount
        elif cmd.action == CharacterCommand.Action.increaseby:
            indicator[cmd.indicator] += cmd.amount
        elif cmd.action == CharacterCommand.Action.decreaseby:
            indicator[cmd.indicator] -= cmd.amount
        else:
            raise ValueError

        self._health, self._power = indicator.values()
        

    def process_query(self, q: CharacterQuery):
        if q.indicator == self.CharacterIndicator.Health:
            return self._health
        elif q.indicator == self.CharacterIndicator.Power:
            return self._power
        else:
            raise ValueError

    @property
    def health(self):
        return self.process_query(CharacterQuery(self.CharacterIndicator.Health))

    @health.setter
    def health(self, amount):
        self.process_command(CharacterCommand(CharacterCommand.Action.setby, self.CharacterIndicator.Health, amount))
    
    @property
    def power(self):
        return self.process_query(CharacterQuery(self.CharacterIndicator.Po))

    @health.setter
    def power(self, amount):
        self.process_command(CharacterCommand(CharacterCommand.Action.setby, self.CharacterIndicator.Power, amount))

    def __str__(self):
        return f"Character {self._name} has Health: {self._health}, Power: {self._power}"

In [171]:
player = Character("ninja", 100, 100)
print(player)
player.health = 30
print(player)

print(player.health)

Character ninja has Health: 100, Power: 100
Character ninja has Health: 30, Power: 100
30
